In [16]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import uic
from Kiwoom import *
import time

form_class = uic.loadUiType("pytrader.ui")[0]

class MyWindow(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)

        self.trade_stocks_done = False

        self.kiwoom = Kiwoom()
        self.kiwoom.comm_connect()

        self.timer = QTimer(self)
        self.timer.start(1000)
        self.timer.timeout.connect(self.timeout)

        self.timer2 = QTimer(self)
        self.timer2.start(1000 *10)
        self.timer2.timeout.connect(self.timeout2)

        accouns_num = int(self.kiwoom.get_login_info("ACCOUNT_CNT"))
        accounts = self.kiwoom.get_login_info("ACCNO")

        accounts_list = accounts.split(';')[0:accouns_num]
        self.comboBox.addItems(accounts_list)

        self.lineEdit.textChanged.connect(self.code_changed)
        self.pushButton.clicked.connect(self.send_order)
        self.pushButton_2.clicked.connect(self.check_balance)

        self.load_buy_sell_list()

    def trade_stocks(self):
        hoga_lookup = {'지정가': "00", '시장가': "03"}

        f = open("buy_list.txt", 'rt')
        buy_list = f.readlines()
        f.close()

        f = open("sell_list.txt", 'rt')
        sell_list = f.readlines()
        f.close()

        # account
        account = self.comboBox.currentText()

        # buy list
        for row_data in buy_list:
            split_row_data = row_data.split(';')
            hoga = split_row_data[2]
            code = split_row_data[1]
            num = split_row_data[3]
            price = split_row_data[4]

            if split_row_data[-1].rstrip() == '매수전':
                self.kiwoom.send_order("send_order_req", "0101", account, 1, code, num, price, hoga_lookup[hoga], "")

        # sell list
        for row_data in sell_list:
            split_row_data = row_data.split(';')
            hoga = split_row_data[2]
            code = split_row_data[1]
            num = split_row_data[3]
            price = split_row_data[4]

            if split_row_data[-1].rstrip() == '매도전':
                self.kiwoom.send_order("send_order_req", "0101", account, 2, code, num, price, hoga_lookup[hoga], "")

        # buy list
        for i, row_data in enumerate(buy_list):
            buy_list[i] = buy_list[i].replace("매수전", "주문완료")

        # file update
        f = open("buy_list.txt", 'wt')
        for row_data in buy_list:
            f.write(row_data)
        f.close()

        # sell list
        for i, row_data in enumerate(sell_list):
            sell_list[i] = sell_list[i].replace("매도전", "주문완료")

        # file update
        f = open("sell_list.txt", 'wt')
        for row_data in sell_list:
            f.write(row_data)
        f.close()

    def load_buy_sell_list(self):
        f = open("buy_list.txt", 'rt')
        buy_list = f.readlines()
        f.close()

        f = open("sell_list.txt", 'rt')
        sell_list = f.readlines()
        f.close()

        row_count = len(buy_list) + len(sell_list)
        self.tableWidget_3.setRowCount(row_count)

        # buy list
        for j in range(len(buy_list)):
            row_data = buy_list[j]
            split_row_data = row_data.split(';')
            split_row_data[1] = self.kiwoom.get_master_code_name(split_row_data[1].rsplit())

            for i in range(len(split_row_data)):
                item = QTableWidgetItem(split_row_data[i].rstrip())
                item.setTextAlignment(Qt.AlignVCenter | Qt.AlignCenter)
                self.tableWidget_3.setItem(j, i, item)

        # sell list
        for j in range(len(sell_list)):
            row_data = sell_list[j]
            split_row_data = row_data.split(';')
            split_row_data[1] = self.kiwoom.get_master_code_name(split_row_data[1].rstrip())

            for i in range(len(split_row_data)):
                item = QTableWidgetItem(split_row_data[i].rstrip())
                item.setTextAlignment(Qt.AlignVCenter | Qt.AlignCenter)
                self.tableWidget_3.setItem(len(buy_list) + j, i, item)

        self.tableWidget_3.resizeRowsToContents()

    def code_changed(self):
        code = self.lineEdit.text()
        name = self.kiwoom.get_master_code_name(code)
        self.lineEdit_2.setText(name)

    def send_order(self):    ## 수동주문 방식
        order_type_lookup = {'신규매수': 1, '신규매도': 2, '매수취소': 3, '매도취소': 4}
        hoga_lookup = {'지정가': "00", '시장가': "03"}

        account = self.comboBox.currentText()
        order_type = self.comboBox_2.currentText()
        code = self.lineEdit.text()
        hoga = self.comboBox_3.currentText()
        num = self.spinBox.value()
        price = self.spinBox_2.value()

        self.kiwoom.send_order("send_order_req", "0101", account, order_type_lookup[order_type], code, num, price, hoga_lookup[hoga], "")

    def timeout(self):
        market_start_time = QTime(9, 0, 0)
        current_time = QTime.currentTime()

        if current_time > market_start_time and self.trade_stocks_done is False:
            self.trade_stocks()
            self.trade_stocks_done = True

        text_time = current_time.toString("hh:mm:ss")
        time_msg = "현재시간: " + text_time

        state = self.kiwoom.get_connect_state()
        if state == 1:
            state_msg = "서버 연결 중"
        else:
            state_msg = "서버 미 연결 중"

        self.statusbar.showMessage(state_msg + " | " + time_msg)

    def timeout2(self):
        if self.checkBox.isChecked():
            self.check_balance()

    def check_balance(self):
        self.kiwoom.reset_opw00018_output()
        account_number = self.kiwoom.get_login_info("ACCNO")
        account_number = account_number.split(';')[0]

        self.kiwoom.set_input_value("계좌번호", account_number)
        self.kiwoom.comm_rq_data("opw00018_req", "opw00018", 0, "2000")

        while self.kiwoom.remained_data:
            time.sleep(0.2)
            self.kiwoom.set_input_value("계좌번호", account_number)
            self.kiwoom.comm_rq_data("opw00018_req", "opw00018", 2, "2000")

        # opw00001
        self.kiwoom.set_input_value("계좌번호", account_number)
        self.kiwoom.comm_rq_data("opw00001_req", "opw00001", 0, "2000")

        # balance
        item = QTableWidgetItem(self.kiwoom.d2_deposit)
        item.setTextAlignment(Qt.AlignVCenter | Qt.AlignRight)
        self.tableWidget.setItem(0, 0, item)

        for i in range(1, 6):
            item = QTableWidgetItem(self.kiwoom.opw00018_output['single'][i - 1])
            item.setTextAlignment(Qt.AlignVCenter | Qt.AlignRight)
            self.tableWidget.setItem(0, i, item)

        self.tableWidget.resizeRowsToContents()

        # Item list
        item_count = len(self.kiwoom.opw00018_output['multi'])
        self.tableWidget_2.setRowCount(item_count)

        for j in range(item_count):
            row = self.kiwoom.opw00018_output['multi'][j]
            for i in range(len(row)):
                item = QTableWidgetItem(row[i])
                item.setTextAlignment(Qt.AlignVCenter | Qt.AlignRight)
                self.tableWidget_2.setItem(j, i, item)

        self.tableWidget_2.resizeRowsToContents()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()

connected
0
0091567
LG전자                                  
3
0
0
0091567
LG전자                                  
3
0
1

LG전자                                  


0
0091567
LG전자                                  
3
0
1

LG전자                                  




In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import uic
from Kiwoom import *
import time
def change_format(data):
    strip_data = data.lstrip('-0')
    if strip_data == '':
        strip_data = '0'

    format_data = format(int(float(strip_data)), ',d')
    if data.startswith('-'):
        format_data = '-' + format_data

    return format_data

In [15]:
change_format('3.89')

'3'

In [ ]:
int(float('1.0000'))

In [ ]:
import sys
from PyQt5.QtWidgets import *
import Kiwoom
import time
from pandas import DataFrame
import datetime
import webreader
import numpy as np

MARKET_KOSPI   = 0
MARKET_KOSDAQ  = 10

class PyMon:
    def __init__(self):
        self.kiwoom = Kiwoom.Kiwoom()
        self.kiwoom.comm_connect()
        self.get_code_list()

    def get_code_list(self):
        self.kospi_codes = self.kiwoom.get_code_list_by_market(MARKET_KOSPI)
        self.kosdaq_codes = self.kiwoom.get_code_list_by_market(MARKET_KOSDAQ)

    def get_ohlcv(self, code, start):
        self.kiwoom.ohlcv = {'date': [], 'open': [], 'high': [], 'low': [], 'close': [], 'volume': []}

        self.kiwoom.set_input_value("종목코드", code)
        self.kiwoom.set_input_value("기준일자", start)
        self.kiwoom.set_input_value("수정주가구분", 1)
        self.kiwoom.comm_rq_data("opt10081_req", "opt10081", 0, "0101")
        time.sleep(0.2)

        df = DataFrame(self.kiwoom.ohlcv, columns=['open', 'high', 'low', 'close', 'volume'],
                       index=self.kiwoom.ohlcv['date'])
        return df

    def check_speedy_rising_volume(self, code):
        today = datetime.datetime.today().strftime("%Y%m%d")
        df = self.get_ohlcv(code, today)
        volumes = df['volume']

        if len(volumes) < 21:
            return False

        sum_vol20 = 0
        today_vol = 0

        for i, vol in enumerate(volumes):
            if i == 0:
                today_vol = vol
            elif 1 <= i <= 20:
                sum_vol20 += vol
            else:
                break

        avg_vol20 = sum_vol20 / 20
        if today_vol > avg_vol20 * 10:
            return True

    def update_buy_list(self, buy_list):
        f = open("buy_list.txt", "wt")
        for code in buy_list:
            f.writelines("매수;" + code + ";시장가;10;0;매수전\n")
        f.close()

    def calculate_estimated_dividend_to_treasury(self, code):
        estimated_dividend_yield = webreader.get_estimated_dividend_yield(code)
        if np.isnan(estimated_dividend_yield):
            estimated_dividend_yield = webreader.get_dividend_yield(code)

            if estimated_dividend_yield == "":
                estimated_dividend_yield = 0

        current_3year_treasury = webreader.get_current_3year_treasury()
        estimated_dividend_to_treasury = float(estimated_dividend_yield) / float(current_3year_treasury)
        return estimated_dividend_to_treasury

    def get_min_max_dividend_to_treasury(self, code):
        previous_dividend_yield = webreader.get_previous_dividend_yield(code)
        three_years_treasury = webreader.get_3year_treasury()
        now = datetime.datetime.now()
        cur_year = now.year
        previous_dividend_to_treasury = {}

        for year in range(cur_year - 5, cur_year):
            if year in previous_dividend_yield.keys() and year in three_years_treasury.keys():
                ratio = float(previous_dividend_yield[year]) / float(three_years_treasury[year])
                previous_dividend_to_treasury[year] = ratio

        print(code, previous_dividend_to_treasury)
        min_ratio = min(previous_dividend_to_treasury.values())
        max_ratio = max(previous_dividend_to_treasury.values())

        return (min_ratio, max_ratio)

    def buy_check_by_dividend_algorithm(self, code):
        estimated_dividend_to_treasury = self.calculate_estimated_dividend_to_treasury(code)
        (min_ratio, max_ratio) = self.get_min_max_dividend_to_treasury(code)

        if estimated_dividend_to_treasury >= max_ratio and max_ratio != 0:
            return (1, estimated_dividend_to_treasury)
        else:
            return (0, estimated_dividend_to_treasury)


    def run_dividend(self):
        buy_list = []

        for code in self.kospi_codes[0:1000]:
            time.sleep(0.5)
            ret = self.buy_check_by_dividend_algorithm(code)
            if ret[0] == 1:
                buy_list.append((code, ret[1]))

        sorted_list = sorted(buy_list, key=lambda t: t[1], reverse=True)

        buy_list = []
        for i in range(0, 5):
            code = sorted_list[i][0]
            buy_list.append(code)

        self.update_buy_list(buy_list)

    def run_dividend_run(self):
        buy_list = []
        buy_list2 = []
        for code in self.kospi_codes[0:1000]:
            time.sleep(0.5)
            ret = self.buy_check_by_dividend_algorithm(code)
            if ret[0] == 1:
                buy_list.append((code, ret[1]))
            elif self.check_speedy_rising_volume(code):
                buy_list2.append(code)
                
        sorted_list = sorted(buy_list, key=lambda t: t[1], reverse=True)

        buy_list = []
        for i in range(0, 5):
            code = sorted_list[i][0]
            buy_list.append(code)
        for i in range(len(buy_list2)) :
            code = buy_list2[i]
            buy_list.append(code)

        self.update_buy_list(buy_list)
        
    def run(self):
        buy_list = []
        num = len(self.kospi_codes[0:1000])

        for i, code in enumerate(self.kospi_codes[0:1000]):
            print(i, '/', num)
            if self.check_speedy_rising_volume(code):
                buy_list.append(code)

        self.update_buy_list(buy_list)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    pymon = PyMon()
    pymon.run_dividend_run()
    #print(pymon.calculate_estimated_dividend_to_treasury('058470'))
    #print(pymon.get_min_max_dividend_to_treasury("058470"))
    #print(pymon.buy_check_by_dividend_algorithm("058470"))


In [ ]:
import urllib.request
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import numpy as np
def get_annual_dividend_yield(code):
    url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701=" % (
        code)
    f = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(f, 'html5lib')
    now = datetime.datetime.now()
    cur_year = now.year
    nan_previous_dividend_yield = {}
    nan_previous_dividend = [np.nan] * 8
    previous_dividend = [np.nan] * 8
    for i, year in enumerate(range(cur_year - 5, cur_year + 3)):
        nan_previous_dividend_yield[year] = nan_previous_dividend[i]

    table_data = soup.find_all('table', {'class': "us_table_ty1 h_fix zigbg_no"})
    if len(table_data) < 5 :
        return  pd.Series(nan_previous_dividend_yield)
    elif len(table_data) >=5 : 
        tr_data = table_data[5].find_all('tr')[-1]
        if len(tr_data) < 1 :
            return  pd.Series(nan_previous_dividend_yield)
        elif len(tr_data) >= 1 :
            td_data = tr_data.find_all('td')
            if len(td_data) <1 :
                return pd.Series(nan_previous_dividend_yield)
            elif len(td_data) >= 1 :
                for i in range(len(td_data)):
                    td_text = td_data[i].get_text()
                    if td_text == '\xa0':
                        td_text = np.nan
                    elif td_text != '\xa0':
                        td_text = float(td_text)
                    else :
                        td_text = np.nan
                    previous_dividend[i] = td_text

    previous_dividend_yield = {}

    for i, year in enumerate(range(cur_year - 5, cur_year + 3)):
        if np.isnan(previous_dividend[i]) == 0 :
            previous_dividend_yield[year] = previous_dividend[i]
        elif np.isnan(previous_dividend[i]) == 1 :
            previous_dividend_yield[year] = np.nan
        else :
            previous_dividend_yield[year] = np.nan

    return pd.Series(previous_dividend_yield)

In [ ]:
get_annual_dividend_yield('001000')

In [ ]:
now = datetime.datetime.now()
cur_year = now.year
url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A000010&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701="
nan_previous_dividend_yield = {}
nan_previous_dividend = [np.nan] * 8
for i, year in enumerate(range(cur_year - 5, cur_year + 3)):
    nan_previous_dividend_yield[year] = nan_previous_dividend[i]
f = urllib.request.urlopen(url).read()
soup = BeautifulSoup(f, 'html5lib')
table_data = soup.find_all('table', {'class': "us_table_ty1 h_fix zigbg_no"})
if len(table_data) < 5 :
    return nan_previous_dividend_yield
elif len(table_data) >=5 : 
    tr_data = table_data[5].find_all('tr')[-1]
    if len(tr_data) < 1 :
        return nan_previous_dividend_yield
    elif len(tr_data) > =1 :
        td_data = tr_data.find_all('td')
        if len(td_data) <1 :
            return nan_previous_dividend_yield
        elif len(td_data) > 1 :
            for i in range(len(td_data)):
                td_text = td_data[i].get_text()
                if td_text == '\xa0':
                    td_text = np.nan
                elif td_text != '\xa0':
                    td_text = float(td_text)
                else :
                    td_text = np.nan
                previous_dividend[i] = td_text

In [ ]:
table_data

In [ ]:
import sys
from PyQt5.QtWidgets import *
import Kiwoom
import time
from pandas import DataFrame
import datetime
import webreader
import numpy as np
import re

class PySell:
    def __init__(self):
        self.kiwoom = Kiwoom.Kiwoom()
        self.kiwoom.comm_connect()
        self.get_bought_code_list()

    def get_bought_code_list(self):
        f = open("buy_list.txt", 'rt')
        buy_list = f.readlines()
        f.close()
        self.bought_list = []
        for i in range(0, 5) :
            bought = re.split(';', buy_list[i])
            if bought[-1]  == '주문완료\n' :
                self.bought_list.append(bought[1])
                
    def get_ohlcv(self, code, start):
        self.kiwoom.ohlcv = {'date': [], 'open': [], 'high': [], 'low': [], 'close': [], 'volume': []}

        self.kiwoom.set_input_value("종목코드", code)
        self.kiwoom.set_input_value("기준일자", start)
        self.kiwoom.set_input_value("수정주가구분", 1)
        self.kiwoom.comm_rq_data("opt10081_req", "opt10081", 0, "0101")
        time.sleep(0.2)

        df = DataFrame(self.kiwoom.ohlcv, columns=['open', 'high', 'low', 'close', 'volume'],
                       index=self.kiwoom.ohlcv['date'])
        return df

    def check_speedy_decreasing_volume(self, code):
        today = datetime.datetime.today().strftime("%Y%m%d")
        df = self.get_ohlcv(code, today)
        volumes = df['volume']

        if len(volumes) < 21:
            return False

        sum_vol20 = 0
        today_vol = 0

        for i, vol in enumerate(volumes):
            if i == 0:
                today_vol = vol
            elif 1 <= i <= 20:
                sum_vol20 += vol
            else:
                break

        avg_vol20 = sum_vol20 / 20
        if today_vol < avg_vol20 / 2:    ## 20일 평균보다 절반 떨어졌을 경우
            return True
        
    def update_sell_list(self, sell_list):
        f = open("sell_list.txt", "wt")
        for code in sell_list:
            f.writelines("매도;", code, ";시장가;10;0;매도전")
        f.close()
        
    def run(self):
        sell_list = []
        num = len(self.bought_list)

        for i, code in enumerate(self.bought_list):
            print(code, i, '/', num)
            if self.check_speedy_decreasing_volume(code):
                sell_list.append(code)

        self.update_sell_list(sell_list)
                
if __name__ == "__main__":
    app = QApplication(sys.argv)
    pysell = PySell()
    pysell.run()

In [ ]:
get_code_list_by_market(0)